<a href="https://colab.research.google.com/github/Welwi/RL_typo/blob/master/RL_typo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this project, I am training a RL algorithm that will master two varying envs with varying complexity.

ENV 1: Cartpole
The goal is to balance a pole, portruding from a cart, in an upright position by only movign the base left or right. This is an env with a low-dimensional observation space.

ENV2: Pong
The goal is to beat the competition. The env has a high-dimensional observation space - learning directly from raw pixels.

In [2]:

!apt-get install -y xvfb python-opengl x11-utils > /dev/null 2>&1
!pip install gym pyvirtualdisplay scikit-video > /dev/null 2>&1

!pip install mitdeeplearning

     |████████████████████████████████| 2.1MB 4.6MB/s 
  Created wheel for mitdeeplearning: filename=mitdeeplearning-0.1.2-cp36-none-any.whl size=2114586 sha256=48db7795b5cf778f217b4f026667ae7f9a343053121a85a2c242658669a21a0a
  Stored in directory: /root/.cache/pip/wheels/27/e1/73/5f01c787621d8a3c857f59876c79e304b9b64db9ff5bd61b74
Successfully built mitdeeplearning


## Steps of RL probs in general:
1. Initialize the env and the agent: describe the different observations and actions the agent can make in the env.

2. Define the agent's memory: this will enable the agent to remember its past actions, observations and rewards

3. Define a reward function: describes the reward associated with an action or sequence of actions

4. Define the learning algorithm: this is used to reinforce the agent's good behavior and discourage the bad behaviors.

In [3]:
import tensorflow as tf
import numpy as np

import base64, io, time, gym
import IPython, functools

import matplotlib.pyplot as plt

from tqdm import tqdm

import mitdeeplearning as mdl